Install Dependancy

In [ ]:
pip install pyproj
pip install osgeo

Import Liabray

In [13]:
import os
from os import path
from osgeo import gdal,ogr,osr
import pyproj
from pyproj import Proj

#If running on Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


function to itrate over folders and identify files

In [5]:
def listAllfiles(path,listfiles):
  path=str(path)
  filename=os.listdir(path)
  for file in filename:
    file = os.path.join(path,file)
    #print(file)
    if os.path.isfile(file)==True:
      #print(file,os.path.isdir(file))
      listfiles.append(file)
    else:
      #print(file,os.path.isdir(file))
      listAllfiles(file,listfiles)
  return listfiles

Parent folder on which the function will be applied

In [6]:
path="/content/drive/My Drive/iirs/data/Exercise 5"
listoffiles=[]
listAllfiles(path,listoffiles)

['/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Raster Analysis/dem/height.img',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Raster Analysis/dem/height.img.aux.xml',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Raster Analysis/dem/height.img.xml',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Raster Analysis/dem/height.rrd',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Raster Analysis/landuse.dbf',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Raster Analysis/landuse.prj',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Raster Analysis/landuse.qpj',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Raster Analysis/landuse.shp',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Raster Analysis/landuse.shx',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Raster Analysis/road.prj',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Raster Analysis/road.dbf',
 '/content/drive/My Drive

File sorting according to their types

In [9]:
rasterFormat=['.img', '.tiff','.grid','.jp2','.bmp','.bil','.bip','.bsq','.dat']
vectorFormat=['.shp', '.geojson','.gml','.kml','.kmz','.gpx','.vct','.vdc','.osm','.dlg']
documentsFormat=['.pdf','.docx','.doc','.txt']
mediaFormat=['.jpeg','.jpg','.svg','.png','.PNG','.mp4','.mp3']
compressedFormat=['.zip']
filesFormat=['.apk']

rasterFiles=[]
vectorFiles=[]
documentFiles=[]
mediaFiles=[]
compressedFiles=[]

for filepath in listoffiles:
    file=filepath.split("/")[-1]
    if os.path.splitext(file)[1] in rasterFormat:
      rasterFiles.append(filepath)
    if os.path.splitext(file)[1] in vectorFormat:
      vectorFiles.append(filepath)
    if os.path.splitext(file)[1] in documentsFormat:
      documentFiles.append(filepath)
    if os.path.splitext(file)[1] in mediaFormat:
      mediaFiles.append(filepath)
    if os.path.splitext(file)[1] in filesFormat:
      compressedFiles.append(filepath)

In [11]:
vectorFiles

['/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Raster Analysis/landuse.shp',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Raster Analysis/road.shp',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Vector Analysis/college_inter/college_inter.shp',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Vector Analysis/residential_density/res_highest_desity.shp',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Vector Analysis/residential_density/res_high_density.shp',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Vector Analysis/residential_density/res_med_density.shp',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Vector Analysis/residential_density/res_min_density.shp',
 '/content/drive/My Drive/iirs/data/Exercise 5/Ex_5 Data/Vector Analysis/roads/road.shp']

Raster Metadata Extraction and Writting

In [14]:
for image in rasterFiles:
  #path="/content/20191020_subset.img" 
  #path="/content/drive/My Drive/Dehradun_2019_rf.tif"
  path=image
  fileName=path.split("/")[-1].split(".")[0]
  dataSource= gdal.Open(path)

  #File size attribute
  fileSizeInB=os.stat(path).st_size
  #print ('file size is ',fileSizeInB)

  #File format attribute
  driver=dataSource.GetDriver()
  fileType=dataSource.GetDriver().LongName
  #print('File format is ',fileType)

  

  #file projection and epsg code
  fileProjectionWKT=dataSource.GetProjection()
  projection_split=fileProjectionWKT.split(",")
  pro=projection_split[0].split('"')
  fileProjection=pro[1]
  spatialRef = osr.SpatialReference(wkt=fileProjectionWKT)
  epsg=spatialRef.GetAuthorityCode(None)
  #print('ile projection is ',fileProjection,' and epsg code is', epsg)


  def GetExtent(gt,cols,rows):
      ext=[]
      xarr=[0,cols]
      yarr=[0,rows]

      for px in xarr:
          for py in yarr:
              x=gt[0]+(px*gt[1])+(py*gt[2])
              y=gt[3]+(px*gt[4])+(py*gt[5])
              ext.append([x,y])
              
          yarr.reverse()
      return ext

  #Raster rows colum and bands
  gt=dataSource.GetGeoTransform()
  cols = dataSource.RasterXSize
  rows = dataSource.RasterYSize
  bands=dataSource.RasterCount
  #print("Raster Size is colum X Row X Bands : {} x {} x {}".format(cols,rows,bands))

  #bounding box coordinates in wgs84
  ext=GetExtent(gt,cols,rows)
  wgs84=pyproj.CRS("EPSG:4326")
  x_min, x_max, y_min, y_max = ext[0][0],ext[2][0],ext[1][1],ext[0][1]
  fileProj4=spatialRef.ExportToProj4()
  filecoordinate=pyproj.CRS(fileProj4)
  West,South =pyproj.transform(filecoordinate,wgs84,x_min,y_min)
  East,North =pyproj.transform(filecoordinate,wgs84,x_max, y_max)
  #print("The bounding box coordinates are ", North,East,South, West)
  #xml hardcoding
  #(fileProjection,epsg,fileName,fileType, West,East,South,North,fileType,fileSizeInB)
  
  import sys 
  stdoutOrigin=sys.stdout
  pathDir=os.path.dirname(image)
  xmlfile_path= pathDir+fileName +".xml"
  sys.stdout = open(xmlfile_path, "w")

  #this xml is created with geonetwork
  print("""
    <gmd:MD_Metadata xmlns:gts="http://www.isotc211.org/2005/gts"
                  xmlns:gml="http://www.opengis.net/gml/3.2"
                  xmlns:gsr="http://www.isotc211.org/2005/gsr"
                  xmlns:gmi="http://www.isotc211.org/2005/gmi"
                  xmlns:gmd="http://www.isotc211.org/2005/gmd"
                  xmlns:gmx="http://www.isotc211.org/2005/gmx"
                  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
                  xmlns:srv="http://www.isotc211.org/2005/srv"
                  xmlns:gco="http://www.isotc211.org/2005/gco"
                  xmlns:xlink="http://www.w3.org/1999/xlink">
    <gmd:fileIdentifier>
        <gco:CharacterString>265d880e-54c1-4346-89a6-320b5871d735</gco:CharacterString>
    </gmd:fileIdentifier>
    <gmd:language>
        <gmd:LanguageCode codeList="http://www.loc.gov/standards/iso639-2/" codeListValue="eng"/>
    </gmd:language>
    <gmd:characterSet>
        <gmd:MD_CharacterSetCode codeListValue="utf8"
                                codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_CharacterSetCode"/>
    </gmd:characterSet>
    <gmd:dateStamp>
        <gco:DateTime>2020-08-12T05:13:32</gco:DateTime>
    </gmd:dateStamp>
    <gmd:metadataStandardName>
        <gco:CharacterString>GSL IIHS</gco:CharacterString>
    </gmd:metadataStandardName>
    <gmd:metadataStandardVersion>
        <gco:CharacterString>0.1</gco:CharacterString>
    </gmd:metadataStandardVersion>
    <gmd:dataSetURI gco:nilReason="missing">
        <gco:CharacterString/>
    </gmd:dataSetURI>
    <gmd:spatialRepresentationInfo>
        <gmd:MD_GridSpatialRepresentation>
          <gmd:numberOfDimensions>
              <gco:Integer>%s</gco:Integer>
          </gmd:numberOfDimensions>
          <gmd:axisDimensionProperties>
              <gmd:MD_Dimension>
                <gmd:dimensionName>
                    <gmd:MD_DimensionNameTypeCode codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_DimensionNameTypeCode"
                                                  codeListValue="column"/>
                </gmd:dimensionName>
                <gmd:dimensionSize>
                    <gco:Integer>%s</gco:Integer>
                </gmd:dimensionSize>
                <gmd:resolution>
                    <gco:Measure uom="meter"/>
                </gmd:resolution>
              </gmd:MD_Dimension>
          </gmd:axisDimensionProperties>
          <gmd:axisDimensionProperties>
              <gmd:MD_Dimension>
                <gmd:dimensionName>
                    <gmd:MD_DimensionNameTypeCode codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_DimensionNameTypeCode"
                                                  codeListValue="row"/>
                </gmd:dimensionName>
                <gmd:dimensionSize>
                    <gco:Integer>%s</gco:Integer>
                </gmd:dimensionSize>
                <gmd:resolution>
                    <gco:Measure uom="meter"/>
                </gmd:resolution>
              </gmd:MD_Dimension>
          </gmd:axisDimensionProperties>
          <gmd:cellGeometry>
              <gmd:MD_CellGeometryCode codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_CellGeometryCode"
                                      codeListValue="area"/>
          </gmd:cellGeometry>
          <gmd:transformationParameterAvailability gco:nilReason="unknown"/>
        </gmd:MD_GridSpatialRepresentation>
    </gmd:spatialRepresentationInfo>
    <gmd:referenceSystemInfo>
        <gmd:MD_ReferenceSystem>
          <gmd:referenceSystemIdentifier>
              <gmd:RS_Identifier>
                <gmd:code>
                    <gco:CharacterString>%s</gco:CharacterString>
                </gmd:code>
                <gmd:codeSpace>
                    <gco:CharacterString>%s</gco:CharacterString>
                </gmd:codeSpace>
              </gmd:RS_Identifier>
          </gmd:referenceSystemIdentifier>
        </gmd:MD_ReferenceSystem>
    </gmd:referenceSystemInfo>
    <gmd:identificationInfo>
        <gmd:MD_DataIdentification>
          <gmd:citation>
              <gmd:CI_Citation>
                <gmd:title>
                    <gco:CharacterString>%s</gco:CharacterString>
                </gmd:title>
                <gmd:date>
                    <gmd:CI_Date>
                      <gmd:date>
                          <gco:DateTime>2020-08-06T20:45:00</gco:DateTime>
                      </gmd:date>
                      <gmd:dateType>
                          <gmd:CI_DateTypeCode codeListValue="publication"
                                              codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#CI_DateTypeCode"/>
                      </gmd:dateType>
                    </gmd:CI_Date>
                </gmd:date>
                <gmd:edition>
                    <gco:CharacterString>11</gco:CharacterString>
                </gmd:edition>
              </gmd:CI_Citation>
          </gmd:citation>
          <gmd:abstract>
              <gco:CharacterString>Read the abstract and supplemental information
            provided in the Vector template for more details.</gco:CharacterString>
          </gmd:abstract>
          <gmd:purpose gco:nilReason="missing">
              <gco:CharacterString/>
          </gmd:purpose>
          <gmd:status>
              <gmd:MD_ProgressCode codeListValue="completed"
                                  codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_ProgressCode"/>
          </gmd:status>
          <gmd:resourceMaintenance/>
          <gmd:descriptiveKeywords>
              <gmd:MD_Keywords>
                <gmd:keyword gco:nilReason="missing">
                    <gco:CharacterString/>
                </gmd:keyword>
              </gmd:MD_Keywords>
          </gmd:descriptiveKeywords>
          <gmd:spatialResolution>
              <gmd:MD_Resolution>
                <gmd:equivalentScale/>
              </gmd:MD_Resolution>
          </gmd:spatialResolution>
          <gmd:language>
              <gco:CharacterString>eng</gco:CharacterString>
          </gmd:language>
          <gmd:characterSet>
              <gmd:MD_CharacterSetCode codeListValue="utf8"
                                      codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_CharacterSetCode"/>
          </gmd:characterSet>
          <gmd:topicCategory>
              <gmd:MD_TopicCategoryCode>boundaries</gmd:MD_TopicCategoryCode>
          </gmd:topicCategory>
        </gmd:MD_DataIdentification>
    </gmd:identificationInfo>
    <gmd:distributionInfo>
        <gmd:MD_Distribution>
          <gmd:distributionFormat>
              <gmd:MD_Format>
                <gmd:name>
                    <gco:CharacterString>%s</gco:CharacterString>
                </gmd:name>
                <gmd:version gco:nilReason="unknown">
                    <gco:CharacterString>1.0</gco:CharacterString>
                </gmd:version>
              </gmd:MD_Format>
          </gmd:distributionFormat>
          <gmd:distributor/>
          <gmd:transferOptions>
              <gmd:MD_DigitalTransferOptions>
                <gmd:unitsOfDistribution>
                    <gco:CharacterString>KB</gco:CharacterString>
                </gmd:unitsOfDistribution>
                <gmd:transferSize>
                    <gco:Real>%s</gco:Real>
                </gmd:transferSize>
              </gmd:MD_DigitalTransferOptions>
          </gmd:transferOptions>
        </gmd:MD_Distribution>
    </gmd:distributionInfo>
    <gmd:dataQualityInfo>
        <gmd:DQ_DataQuality>
          <gmd:scope>
              <gmd:DQ_Scope>
                <gmd:extent>
                    <gmd:EX_Extent>
                      <gmd:geographicElement>
                          <gmd:EX_GeographicBoundingBox>
                            <gmd:westBoundLongitude>
                                <gco:Decimal>%s</gco:Decimal>
                            </gmd:westBoundLongitude>
                            <gmd:eastBoundLongitude>
                                <gco:Decimal>%s</gco:Decimal>
                            </gmd:eastBoundLongitude>
                            <gmd:southBoundLatitude>
                                <gco:Decimal>%s</gco:Decimal>
                            </gmd:southBoundLatitude>
                            <gmd:northBoundLatitude>
                                <gco:Decimal>%s</gco:Decimal>
                            </gmd:northBoundLatitude>
                          </gmd:EX_GeographicBoundingBox>
                      </gmd:geographicElement>
                      <gmd:temporalElement>
                          <gmd:EX_TemporalExtent>
                            <gmd:extent>
                                <gml:TimePeriod gml:id="d167988e159a1053982">
                                  <gml:beginPosition>2020-08-01</gml:beginPosition>
                                  <gml:endPosition>2020-08-06</gml:endPosition>
                                </gml:TimePeriod>
                            </gmd:extent>
                          </gmd:EX_TemporalExtent>
                      </gmd:temporalElement>
                      <gmd:verticalElement>
                          <gmd:EX_VerticalExtent>
                            <gmd:minimumValue>
                                <gco:Real>10</gco:Real>
                            </gmd:minimumValue>
                            <gmd:maximumValue>
                                <gco:Real>0</gco:Real>
                            </gmd:maximumValue>
                          </gmd:EX_VerticalExtent>
                      </gmd:verticalElement>
                    </gmd:EX_Extent>
                </gmd:extent>
              </gmd:DQ_Scope>
          </gmd:scope>
        </gmd:DQ_DataQuality>
    </gmd:dataQualityInfo>
    <gmd:metadataConstraints>
        <gmd:MD_LegalConstraints>
          <gmd:accessConstraints>
              <gmd:MD_RestrictionCode codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_RestrictionCode"
                                      codeListValue=""/>
          </gmd:accessConstraints>
          <gmd:useConstraints>
              <gmd:MD_RestrictionCode codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_RestrictionCode"
                                      codeListValue=""/>
          </gmd:useConstraints>
        </gmd:MD_LegalConstraints>
    </gmd:metadataConstraints>
  </gmd:MD_Metadata>
  """%(bands,cols,rows,fileProjection,epsg,fileName,fileType,fileSizeInB,West,East,South,North))
  sys.stdout.close()
  sys.stdout=stdoutOrigin

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:56: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1


Vector metadata extraction and writting

In [15]:
for vector in vectorFiles:
  shpFile=vector
  #shpFile = r'/content/drive/My Drive/iirs/data/Exercise 4/Ex_4 Data/inter_colleges.shp' #location to the vector file 

  #File size attribute
  fileSizeInB=os.stat(shpFile).st_size
  #print ('file size is ',fileSizeInB)

  dataSource = ogr.Open(shpFile)

  #File format attribute
  driver=dataSource.GetDriver()
  fileType=driver.GetName()
  #print('File format is ',fileType)

  #file name attribute for key word or tiltle
  layer = dataSource.GetLayer(0)
  fileName=layer.GetName()
  #print('File name is ',fileName)


  #file projection and epsg code
  spatialRef = layer.GetSpatialRef()
  fileProjectionWKT=spatialRef.ExportToWkt()
  projection_split=fileProjectionWKT.split(",")
  pro=projection_split[0].split('"')
  fileProjection=pro[1]
  epsg=spatialRef.GetAuthorityCode(None)
  #print('File projection is ',fileProjection,' and epsg code is', epsg)


  #bounding box coordinates in wgs84
  wgs84=pyproj.CRS("EPSG:4326")
  x_min, x_max, y_min, y_max = layer.GetExtent()
  fileProj4=spatialRef.ExportToProj4()
  filecoordinate=pyproj.CRS(fileProj4)
  West,South =pyproj.transform(filecoordinate,wgs84,x_min,y_min)
  East,North =pyproj.transform(filecoordinate,wgs84,x_max, y_max)

  #print("The bounding box coordinates are ", North,East,South, West)

  #xml hardcoding
  #(fileProjection,epsg,fileName,fileType, West,East,South,North,fileType,fileSizeInB)

  import sys 
  stdoutOrigin=sys.stdout
  pathDir=os.path.dirname(shpFile)
  xmlfile_path= pathDir+fileName +".xml"
  sys.stdout = open(xmlfile_path, "w")

  #this xml is created with geonetwork

  print("""
  <?xml version="1.0" encoding="UTF-8"?>
  <gmd:MD_Metadata xmlns:gmd="http://www.isotc211.org/2005/gmd"
                  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
                  xmlns:gco="http://www.isotc211.org/2005/gco"
                  xmlns:srv="http://www.isotc211.org/2005/srv"
                  xmlns:gmx="http://www.isotc211.org/2005/gmx"
                  xmlns:gts="http://www.isotc211.org/2005/gts"
                  xmlns:gsr="http://www.isotc211.org/2005/gsr"
                  xmlns:gmi="http://www.isotc211.org/2005/gmi"
                  xmlns:gml="http://www.opengis.net/gml/3.2"
                  xmlns:xlink="http://www.w3.org/1999/xlink"
                  xsi:schemaLocation="http://www.isotc211.org/2005/gmd http://schemas.opengis.net/csw/2.0.2/profiles/apiso/1.0.0/apiso.xsd">
    <gmd:fileIdentifier>
        <gco:CharacterString>27d58be6-6665-4a38-b440-b0091b67dd91</gco:CharacterString>
    </gmd:fileIdentifier>
    <gmd:language xmlns:geonet="http://www.fao.org/geonetwork">
        <gmd:LanguageCode codeList="http://www.loc.gov/standards/iso639-2/" codeListValue="eng"/>
    </gmd:language>
    <gmd:characterSet xmlns:geonet="http://www.fao.org/geonetwork">
        <gmd:MD_CharacterSetCode codeListValue="utf8"
                                codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_CharacterSetCode"/>
    </gmd:characterSet>
    <gmd:parentIdentifier xmlns:geonet="http://www.fao.org/geonetwork">
        <gco:CharacterString>asa</gco:CharacterString>
    </gmd:parentIdentifier>
    <gmd:hierarchyLevel xmlns:geonet="http://www.fao.org/geonetwork">
        <gmd:MD_ScopeCode codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_ScopeCode"
                          codeListValue="attribute"/>
    </gmd:hierarchyLevel>
    <gmd:dateStamp xmlns:geonet="http://www.fao.org/geonetwork">
        <gco:DateTime>2020-07-31T04:28:41</gco:DateTime>
    </gmd:dateStamp>
    <gmd:metadataStandardName xmlns:geonet="http://www.fao.org/geonetwork">
        <gco:CharacterString>ISO 19115:2003/19139</gco:CharacterString>
    </gmd:metadataStandardName>
    <gmd:metadataStandardVersion xmlns:geonet="http://www.fao.org/geonetwork">
        <gco:CharacterString>1.0</gco:CharacterString>
    </gmd:metadataStandardVersion>
    <gmd:dataSetURI xmlns:geonet="http://www.fao.org/geonetwork">
        <gco:CharacterString>http://spatialdata.in/geonetwork/srv/eng/catalog.edit#/metadata/430?justcreated&amp;redirectUrl=catalog.edit</gco:CharacterString>
    </gmd:dataSetURI>
    <gmd:spatialRepresentationInfo xmlns:geonet="http://www.fao.org/geonetwork">
        <gmd:MD_Georectified>
          <gmd:numberOfDimensions>
              <gco:Integer>1</gco:Integer>
          </gmd:numberOfDimensions>
          <gmd:cellGeometry>
              <gmd:MD_CellGeometryCode codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_CellGeometryCode"
                                      codeListValue="area"/>
          </gmd:cellGeometry>
          <gmd:transformationParameterAvailability gco:nilReason="unknown"/>
          <gmd:checkPointAvailability gco:nilReason="unknown"/>
          <gmd:pointInPixel>
              <gmd:MD_PixelOrientationCode>center</gmd:MD_PixelOrientationCode>
          </gmd:pointInPixel>
        </gmd:MD_Georectified>
    </gmd:spatialRepresentationInfo>
    <gmd:referenceSystemInfo xmlns:geonet="http://www.fao.org/geonetwork">
        <gmd:MD_ReferenceSystem>
          <gmd:referenceSystemIdentifier>
              <gmd:RS_Identifier>
                <gmd:authority/>
                <gmd:code>
                    <gco:CharacterString>%s</gco:CharacterString>
                </gmd:code>
                <gmd:codeSpace>
                    <gco:CharacterString>EPSG %s</gco:CharacterString>
                </gmd:codeSpace>
                <gmd:version>
                    <gco:CharacterString>1</gco:CharacterString>
                </gmd:version>
              </gmd:RS_Identifier>
          </gmd:referenceSystemIdentifier>
        </gmd:MD_ReferenceSystem>
    </gmd:referenceSystemInfo>
    <gmd:identificationInfo xmlns:geonet="http://www.fao.org/geonetwork">
        <gmd:MD_DataIdentification>
          <gmd:citation>
              <gmd:CI_Citation>
                <gmd:title>
                    <gco:CharacterString>%s</gco:CharacterString>
                </gmd:title>
                <gmd:date>
                    <gmd:CI_Date>
                      <gmd:date>
                          <gco:Date>2020-01</gco:Date>
                      </gmd:date>
                      <gmd:dateType>
                          <gmd:CI_DateTypeCode codeListValue="creation"
                                              codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#CI_DateTypeCode"/>
                      </gmd:dateType>
                    </gmd:CI_Date>
                </gmd:date>
                <gmd:edition>
                    <gco:CharacterString>http://localhost:8080/demodem/editioin</gco:CharacterString>
                </gmd:edition>
              </gmd:CI_Citation>
          </gmd:citation>
          <gmd:abstract>
              <gco:CharacterString>The ISO19115 metadata standard is the preferred metadata standard to use. If unsure what templates to start with, use this one.</gco:CharacterString>
          </gmd:abstract>
          <gmd:purpose>
              <gco:CharacterString>no purpose</gco:CharacterString>
          </gmd:purpose>
          <gmd:status>
              <gmd:MD_ProgressCode codeListValue="completed"
                                  codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_ProgressCode"/>
          </gmd:status>
          <gmd:pointOfContact>
              <gmd:CI_ResponsibleParty>
                <gmd:individualName>
                    <gco:CharacterString>Shubham</gco:CharacterString>
                </gmd:individualName>
                <gmd:organisationName>
                    <gco:CharacterString>IIHS</gco:CharacterString>
                </gmd:organisationName>
                <gmd:positionName gco:nilReason="missing">
                    <gco:CharacterString/>
                </gmd:positionName>
                <gmd:contactInfo>
                    <gmd:CI_Contact>
                      <gmd:address>
                          <gmd:CI_Address>
                            <gmd:electronicMailAddress>
                                <gco:CharacterString>smp4475@gmail.com</gco:CharacterString>
                            </gmd:electronicMailAddress>
                          </gmd:CI_Address>
                      </gmd:address>
                    </gmd:CI_Contact>
                </gmd:contactInfo>
                <gmd:role>
                    <gmd:CI_RoleCode codeListValue="originator"
                                    codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#CI_RoleCode"/>
                </gmd:role>
              </gmd:CI_ResponsibleParty>
          </gmd:pointOfContact>
          <gmd:resourceMaintenance>
              <gmd:MD_MaintenanceInformation>
                <gmd:maintenanceAndUpdateFrequency>
                    <gmd:MD_MaintenanceFrequencyCode codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_MaintenanceFrequencyCode"
                                                    codeListValue="continual"/>
                </gmd:maintenanceAndUpdateFrequency>
              </gmd:MD_MaintenanceInformation>
          </gmd:resourceMaintenance>
          <gmd:resourceFormat>
              <gmd:MD_Format>
                <gmd:name>
                    <gco:CharacterString>%s</gco:CharacterString>
                </gmd:name>
                <gmd:version>
                    <gco:CharacterString>1.1</gco:CharacterString>
                </gmd:version>
                <gmd:formatDistributor>
                    <gmd:MD_Distributor>
                      <gmd:distributorFormat>
                          <gmd:MD_Format>
                            <gmd:name gco:nilReason="missing">
                                <gco:CharacterString/>
                            </gmd:name>
                            <gmd:version gco:nilReason="missing">
                                <gco:CharacterString/>
                            </gmd:version>
                            <gmd:specification gco:nilReason="missing">
                                <gco:CharacterString/>
                            </gmd:specification>
                          </gmd:MD_Format>
                      </gmd:distributorFormat>
                      <gmd:distributorTransferOptions>
                          <gmd:MD_DigitalTransferOptions>
                            <gmd:transferSize>
                                <gco:Real/>
                            </gmd:transferSize>
                          </gmd:MD_DigitalTransferOptions>
                      </gmd:distributorTransferOptions>
                    </gmd:MD_Distributor>
                </gmd:formatDistributor>
              </gmd:MD_Format>
          </gmd:resourceFormat>
          <gmd:descriptiveKeywords>
              <gmd:MD_Keywords>
                <gmd:keyword>
                    <gco:CharacterString>data, data2</gco:CharacterString>
                </gmd:keyword>
              </gmd:MD_Keywords>
          </gmd:descriptiveKeywords>
          <gmd:resourceSpecificUsage>
              <gmd:MD_Usage>
                <gmd:specificUsage>
                    <gco:CharacterString>not specific</gco:CharacterString>
                </gmd:specificUsage>
                <gmd:usageDateTime>
                    <gco:Date>2020-07</gco:Date>
                </gmd:usageDateTime>
                <gmd:userDeterminedLimitations>
                    <gco:CharacterString>no</gco:CharacterString>
                </gmd:userDeterminedLimitations>
              </gmd:MD_Usage>
          </gmd:resourceSpecificUsage>
          <gmd:resourceConstraints>
              <gmd:MD_LegalConstraints>
                <gmd:accessConstraints>
                    <gmd:MD_RestrictionCode codeListValue="intellectualPropertyRights"
                                            codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_RestrictionCode"/>
                </gmd:accessConstraints>
                <gmd:useConstraints>
                    <gmd:MD_RestrictionCode codeListValue="patent"
                                            codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_RestrictionCode"/>
                </gmd:useConstraints>
              </gmd:MD_LegalConstraints>
          </gmd:resourceConstraints>
          <gmd:aggregationInfo>
              <gmd:MD_AggregateInformation>
                <gmd:associationType>
                    <gmd:DS_AssociationTypeCode codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#DS_AssociationTypeCode"
                                                codeListValue="source"/>
                </gmd:associationType>
              </gmd:MD_AggregateInformation>
          </gmd:aggregationInfo>
          <gmd:spatialRepresentationType>
              <gmd:MD_SpatialRepresentationTypeCode codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_SpatialRepresentationTypeCode"
                                                    codeListValue="vector"/>
          </gmd:spatialRepresentationType>
          <gmd:language>
              <gmd:LanguageCode codeList="http://www.loc.gov/standards/iso639-2/" codeListValue="eng"/>
          </gmd:language>
          <gmd:characterSet>
              <gmd:MD_CharacterSetCode codeListValue="utf8"
                                      codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_CharacterSetCode"/>
          </gmd:characterSet>
          <gmd:topicCategory>
              <gmd:MD_TopicCategoryCode>boundaries</gmd:MD_TopicCategoryCode>
          </gmd:topicCategory>
          <gmd:topicCategory>
              <gmd:MD_TopicCategoryCode>environment</gmd:MD_TopicCategoryCode>
          </gmd:topicCategory>
          <gmd:extent>
              <gmd:EX_Extent>
                <gmd:geographicElement>
                    <gmd:EX_GeographicBoundingBox>
                      <gmd:westBoundLongitude>
                          <gco:Decimal>%s</gco:Decimal>
                      </gmd:westBoundLongitude>
                      <gmd:eastBoundLongitude>
                          <gco:Decimal>%s</gco:Decimal>
                      </gmd:eastBoundLongitude>
                      <gmd:southBoundLatitude>
                          <gco:Decimal>%s</gco:Decimal>
                      </gmd:southBoundLatitude>
                      <gmd:northBoundLatitude>
                          <gco:Decimal>%s</gco:Decimal>
                      </gmd:northBoundLatitude>
                    </gmd:EX_GeographicBoundingBox>
                </gmd:geographicElement>
                <gmd:verticalElement/>
              </gmd:EX_Extent>
          </gmd:extent>
          <gmd:supplementalInformation gco:nilReason="missing">
              <gco:CharacterString/>
          </gmd:supplementalInformation>
        </gmd:MD_DataIdentification>
    </gmd:identificationInfo>
    <gmd:contentInfo xmlns:geonet="http://www.fao.org/geonetwork">
        <gmd:MD_CoverageDescription>
          <gmd:attributeDescription>
              <gco:RecordType>aaa</gco:RecordType>
          </gmd:attributeDescription>
          <gmd:contentType>
              <gmd:MD_CoverageContentTypeCode codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_CoverageContentTypeCode"
                                              codeListValue="physicalMeasurement"/>
          </gmd:contentType>
          <gmd:dimension>
              <gmd:MD_RangeDimension>
                <gmd:sequenceIdentifier>
                    <gco:MemberName>
                      <gco:aName>
                          <gco:CharacterString>sas12333shubham</gco:CharacterString>
                      </gco:aName>
                      <gco:attributeType>
                          <gco:TypeName>
                            <gco:aName>
                                <gco:CharacterString>asa123</gco:CharacterString>
                            </gco:aName>
                          </gco:TypeName>
                      </gco:attributeType>
                    </gco:MemberName>
                </gmd:sequenceIdentifier>
                <gmd:descriptor>
                    <gco:CharacterString>asas123</gco:CharacterString>
                </gmd:descriptor>
              </gmd:MD_RangeDimension>
          </gmd:dimension>
        </gmd:MD_CoverageDescription>
    </gmd:contentInfo>
    <gmd:distributionInfo xmlns:geonet="http://www.fao.org/geonetwork">
        <gmd:MD_Distribution>
          <gmd:distributionFormat>
              <gmd:MD_Format>
                <gmd:name>
                    <gco:CharacterString>%s</gco:CharacterString>
                </gmd:name>
                <gmd:version gco:nilReason="unknown">
                    <gco:CharacterString>1</gco:CharacterString>
                </gmd:version>
                <gmd:specification gco:nilReason="missing">
                    <gco:CharacterString/>
                </gmd:specification>
              </gmd:MD_Format>
          </gmd:distributionFormat>
          <gmd:transferOptions>
              <gmd:MD_DigitalTransferOptions>
                <gmd:transferSize>
                    <gco:Real>%d</gco:Real>
                </gmd:transferSize>
              </gmd:MD_DigitalTransferOptions>
          </gmd:transferOptions>
        </gmd:MD_Distribution>
    </gmd:distributionInfo>
    <gmd:dataQualityInfo xmlns:geonet="http://www.fao.org/geonetwork">
        <gmd:DQ_DataQuality>
          <gmd:scope>
              <gmd:DQ_Scope>
                <gmd:level>
                    <gmd:MD_ScopeCode codeListValue="dataset"
                                      codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#MD_ScopeCode"/>
                </gmd:level>
              </gmd:DQ_Scope>
          </gmd:scope>
          <gmd:lineage>
              <gmd:LI_Lineage>
                <gmd:processStep>
                    <gmd:LI_ProcessStep>
                      <gmd:description>
                          <gco:CharacterString>111</gco:CharacterString>
                      </gmd:description>
                      <gmd:dateTime>
                          <gco:Date>2020-12</gco:Date>
                      </gmd:dateTime>
                      <gmd:processor>
                          <gmd:CI_ResponsibleParty>
                            <gmd:individualName>
                                <gco:CharacterString>sp</gco:CharacterString>
                            </gmd:individualName>
                            <gmd:organisationName>
                                <gco:CharacterString>shubham</gco:CharacterString>
                            </gmd:organisationName>
                            <gmd:positionName gco:nilReason="missing">
                                <gco:CharacterString/>
                            </gmd:positionName>
                            <gmd:contactInfo>
                                <gmd:CI_Contact>
                                  <gmd:address>
                                      <gmd:CI_Address>
                                        <gmd:electronicMailAddress>
                                            <gco:CharacterString>smao</gco:CharacterString>
                                        </gmd:electronicMailAddress>
                                      </gmd:CI_Address>
                                  </gmd:address>
                                </gmd:CI_Contact>
                            </gmd:contactInfo>
                            <gmd:role>
                                <gmd:CI_RoleCode codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#CI_RoleCode"
                                                codeListValue="processor"/>
                            </gmd:role>
                          </gmd:CI_ResponsibleParty>
                      </gmd:processor>
                      <gmd:source uuidref="">
                          <gmd:LI_Source>
                            <gmd:description>
                                <gco:CharacterString>sa</gco:CharacterString>
                            </gmd:description>
                            <gmd:sourceCitation>
                                <gmd:CI_Citation>
                                  <gmd:title>
                                      <gco:CharacterString>asdsda</gco:CharacterString>
                                  </gmd:title>
                                  <gmd:date>
                                      <gmd:CI_Date>
                                        <gmd:date>
                                            <gco:Date>2020-08</gco:Date>
                                        </gmd:date>
                                        <gmd:dateType>
                                            <gmd:CI_DateTypeCode codeList="http://standards.iso.org/iso/19139/resources/gmxCodelists.xml#CI_DateTypeCode"
                                                                codeListValue="creation"/>
                                        </gmd:dateType>
                                      </gmd:CI_Date>
                                  </gmd:date>
                                </gmd:CI_Citation>
                            </gmd:sourceCitation>
                          </gmd:LI_Source>
                      </gmd:source>
                    </gmd:LI_ProcessStep>
                </gmd:processStep>
              </gmd:LI_Lineage>
          </gmd:lineage>
        </gmd:DQ_DataQuality>
    </gmd:dataQualityInfo>
  </gmd:MD_Metadata>"""%(fileProjection,epsg,fileName,fileType, West,East,South,North,fileType,fileSizeInB))

  sys.stdout.close()
  sys.stdout=stdoutOrigin

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.h